### Connect to elasticsearch

In [16]:
from elasticsearch import Elasticsearch
import urllib3
import warnings
from flatten_json import flatten
import os
import jinja2
import json
import polars as pl


ELASTIC_PASS = os.environ["ES_PASS"]
NODES = [
    "https://esh1p:9200",
    "https://esh2p:9200",
    
]
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
warnings.simplefilter(action='ignore', category=Warning)



es = Elasticsearch(NODES,
    basic_auth=("elastic", ELASTIC_PASS),
    verify_certs=False                       
)

### Find masquerading processes(https://attack.mitre.org/techniques/T1036/005/) using fuzzy search

In [60]:
environment = jinja2.Environment()


#optional variables to for search
commonly_abused_processes = ["svchost.exe", "explorer.exe", "iexplore.exe"]
index_pattern = "windows"

# create polars dataframe
df = pl.DataFrame()



search_request_template = environment.from_string('''
 {
    
    "bool": {

      "should": [

        {
          "match": { "event.provider": "Microsoft-Windows-Sysmon"  }
        },
    {%for pname in process_name_arr %}
    
        {"fuzzy": {
          "process.name": {
            "value": "{{pname}}"
          }
        }}
        
    {%if process_name_arr.index(pname)  < (process_name_arr | length - 1 ) %} , {%endif%}
    
    {%endfor%}
    ],
    "must_not": [
    
    {%for pname in process_name_arr %}
    
      {
        "term": {
            "process.name": "{{pname}}"
        }
      }
      
    {%if process_name_arr.index(pname)  < (process_name_arr | length - 1 ) %} , {%endif%}  
      
    {%endfor%}

      ] 
    }
  }

''')


search_req_json = json.loads(search_request_template.render(process_name_arr = commonly_abused_processes))
response = es.search(index=index_pattern,
                     query=search_req_json,
                         source = [ "process.name", "process.executable", "process.parent.executable"])


In [62]:
for event in response['hits']['hits']:
    
    
    if "parent" in event['_source']['process'].keys():
        row = pl.DataFrame({ "process_name": event['_source']['process']['name'], "process_executable": event['_source']['process']['executable'] , "parent_executable": event['_source']['process']['parent']['executable']  })
        
    else:
        row = pl.DataFrame({ "process_name": event['_source']['process']['name'], "process_executable": event['_source']['process']['executable'], "parent_executable": "None" })

    
    df = pl.concat([df, row])
    
print(df.unique())

shape: (5, 3)
┌──────────────┬─────────────────────────────────┬─────────────────────────────────────┐
│ process_name ┆ process_executable              ┆ parent_executable                   │
│ ---          ┆ ---                             ┆ ---                                 │
│ str          ┆ str                             ┆ str                                 │
╞══════════════╪═════════════════════════════════╪═════════════════════════════════════╡
│ scvhost.exe  ┆ C:\Users\Filip\dist\scvhost.exe ┆ C:\Windows\System32\WindowsPower... │
│ scvhost.exe  ┆ C:\Users\Filip\dist\scvhost.exe ┆ C:\Users\Filip\dist\scvhost.exe     │
│ scvhost.exe  ┆ C:\Users\Filip\dist\scvhost.exe ┆ None                                │
│ sihost.exe   ┆ C:\Windows\System32\sihost.exe  ┆ C:\Windows\System32\svchost.exe     │
│ sihost.exe   ┆ C:\Windows\System32\sihost.exe  ┆ None                                │
└──────────────┴─────────────────────────────────┴─────────────────────────────────────┘
